# Evaluation of MLP log-likelihood for binary classification

Confirm PyTorch and manually coded MLP log-likelihood coincide

In [1]:
## Import packages

import torch
import torch.nn as nn
import torch.nn.functional as F

from eeyore.models.mlp import Hyperparameters, MLP
from eeyore.data import XOR

## Compute MLP log-likelihood using eeyore API version

In [2]:
## Load XOR data

xor = XOR(dtype=torch.float64)

data = xor.data
labels = xor.labels

In [3]:
## Setup MLP model

hparams = Hyperparameters([2, 2, 1])
model = MLP(hparams=hparams, dtype=torch.float64)

In [4]:
## Fix model parameters

theta = torch.tensor([1.1, -2.9, -0.4, 0.8, 4.3, 9.2, 4.44, -3.4, 7.2], dtype=torch.float64)
model.set_params(theta.clone().detach())

In [5]:
## Compute MLP log-likelihood using eeyore API version

result01 = model.log_lik(data, labels)
result01

tensor(-16.0859, dtype=torch.float64, grad_fn=<NegBackward>)

## Compute MLP log-likelihood using Pytorch loss and cross entropy

In [6]:
## Compute model output using Pytorch forward() method

out = model(data)
# out = model.forward(data)
out

tensor([[0.9997],
        [0.9994],
        [0.9997],
        [0.9996]], dtype=torch.float64, grad_fn=<SigmoidBackward>)

In [7]:
## Define logit loss

criterion = nn.BCEWithLogitsLoss(reduction='sum')
loss = criterion(out, labels)

In [8]:
## Define logit function

def logit(p):
    return torch.log(p/(1-p))

In [9]:
## Compute MLP log-likelihood using Pytorch binary_cross_entropy_with_logits

result02a = -F.binary_cross_entropy_with_logits(logit(out), labels, reduction='sum')
result02a

tensor(-16.0859, dtype=torch.float64, grad_fn=<NegBackward>)

In [10]:
## Compute MLP log-likelihood using Pytorch binary_cross_entropy

result02b = -F.binary_cross_entropy(out, labels, reduction='sum')
result02b

tensor(-16.0859, dtype=torch.float64, grad_fn=<NegBackward>)

## Compute MLP log-likelihood manually given Pytorch forward output

In [11]:
## Define sigmoid function

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [12]:
## Define MLP log-lik

def log_lik(g, y):
    term = y * torch.log(sigmoid(g)) + (1-y) * torch.log(1-sigmoid(g))
    return torch.sum(term)

In [13]:
## Compute MLP log-lik

result03 = log_lik(logit(out), labels)
result03

tensor(-16.0859, dtype=torch.float64, grad_fn=<SumBackward0>)

## Compute MLP log-likelihood similarly to model.log_lik

In [14]:
## Print output from forward pass of MLP model

out

tensor([[0.9997],
        [0.9994],
        [0.9997],
        [0.9996]], dtype=torch.float64, grad_fn=<SigmoidBackward>)

In [15]:
## Define MLP forward01

def forward01(x):
    h = x
    for fc, activation in zip(model.fc_layers, model.hp.activations):
        h = activation(fc(h))
    return h

In [16]:
## Compute MLP forward01

out_of_forward01 = forward01(data)
out_of_forward01

tensor([[0.9997],
        [0.9994],
        [0.9997],
        [0.9996]], dtype=torch.float64, grad_fn=<SigmoidBackward>)

In [17]:
## Compute MLP log-lik given forward01 out

result04 = log_lik(logit(out_of_forward01), labels)
result04

tensor(-16.0859, dtype=torch.float64, grad_fn=<SumBackward0>)

## Compute MLP log-likelihood manually by invoking linear layer and activation functions

In [18]:
## Define MLP forward02

def forward02(x, num_layers):
    h = x
    for i in range(num_layers):
        h = model.hp.activations[i](model.fc_layers[i](h))
    return h

In [19]:
## Compute MLP forward02

out_of_forward02 = forward02(data, 2)
out_of_forward02

tensor([[0.9997],
        [0.9994],
        [0.9997],
        [0.9996]], dtype=torch.float64, grad_fn=<SigmoidBackward>)

In [20]:
## Compute MLP log-lik given forward02 out

result05 = log_lik(logit(out_of_forward02), labels)
result05

tensor(-16.0859, dtype=torch.float64, grad_fn=<SumBackward0>)

## Compute MLP log-likelihood computing manually linear layers given activations

In [21]:
## Define MLP forward03

def forward03(x, num_layers):
    h = x
    for i in range(num_layers):
        h = model.hp.activations[i](h @ model.fc_layers[i].weight.t() + model.fc_layers[i].bias)
    return h

In [22]:
## Compute MLP forward03

out_of_forward03 = forward03(data, 2)
out_of_forward03

tensor([[0.9997],
        [0.9994],
        [0.9997],
        [0.9996]], dtype=torch.float64, grad_fn=<SigmoidBackward>)

In [23]:
## Compute MLP log-lik given forward03 out

result06 = log_lik(logit(out_of_forward03), labels)
result06

tensor(-16.0859, dtype=torch.float64, grad_fn=<SumBackward0>)

## Compute MLP log-likelihood fully manually

In [24]:
## Define MLP forward04

def forward04(x):
    w1 = theta[0:4].view(2, 2)
    b1 = theta[4:6].view(2)
    g1 = x @ w1.t() + b1
    h1 = torch.sigmoid(g1)
    w2 = theta[6:8].view(1, 2)
    b2 = theta[8:9].view(1)
    g2 = h1 @ w2.t() + b2
    h2 = torch.sigmoid(g2)
    
    return h2

In [25]:
## Compute MLP forward04

out_of_forward04 = forward04(data)
out_of_forward04

tensor([[0.9997],
        [0.9994],
        [0.9997],
        [0.9996]], dtype=torch.float64)

In [26]:
## Compute MLP log-lik given forward04 out

result07 = log_lik(logit(out_of_forward04), labels)
result07

tensor(-16.0859, dtype=torch.float64)

## Print out values of all log-lik implementations

In [27]:
[p.data.item() for p in
    [result01, result02a, result02b, result03, result04, result05, result06, result07]
]

[-16.08587869723768,
 -16.08587869723768,
 -16.08587869723768,
 -16.08587869723768,
 -16.08587869723768,
 -16.08587869723768,
 -16.08587869723768,
 -16.08587869723768]